Importações

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pingouin as pg
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

Importar base de dados

In [ ]:
df = pd.read_csv("../databases/ENEM_FULL.csv")
display(df.sample(5))

Funções Auxiliares

In [ ]:
def testar_normalidade_univariada(X:pd.DataFrame):
    print("\nTeste de normalidade univariada (Kolmogorov-Smirnov):")
    resultados = []
    for col in X.columns:
        stat, p = stats.kstest(X[col], 'norm', args=(X[col].mean(), X[col].std(ddof=1)))
        resultados.append((col, p))
        interpret = "Normal" if p > 0.05 else "Não normal"
        print(f"  {col} p={p}: {interpret}")
    return resultados


def testar_normalidade_multivariada(X:pd.DataFrame):
    print("\nTeste de normalidade multivariada (Mardia):")
    res = pg.multivariate_normality(X, alpha=0.05)
    print(f"Normal: {res.normal}")
    print(f"P-value={res.pval:.4f}, Hanze-Zirkler p={res.hz:.4f}")
    return res


def testar_outliers(X:pd.DataFrame):
    print("\nDetecção de outliers (z-score > 3):")
    z_scores = np.abs(stats.zscore(X))
    outliers = (z_scores > 3).any(axis=1)
    qtd = outliers.sum()
    print(f"Outliers detectados: {qtd} observações ({qtd/len(X)})")
    return outliers


def testar_multicolinearidade(X:pd.DataFrame):
    print("\nTeste de multicolinearidade (VIF):")
    X_vif = X.assign(const=1)
    vif_data = pd.DataFrame({
        "Variável": X.columns,
        "VIF": [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1]-1)]
    })
    print(vif_data.to_string(index=False))
    return vif_data


def testar_homogeneidade(X:pd.DataFrame, y:pd.DataFrame):
    print("\nTeste de homogeneidade das matrizes de covariância (Box’s M):")
        
    dvs = ["NU_NOTA_CH","NU_NOTA_CN","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]
    res = pg.box_m(data=df, dvs=dvs, group="TP_DEPENDENCIA_ADM_ESC", alpha=0.05)
    print(res)
    #return res


def testar_linearidade(X:pd.DataFrame):
    print("\nAvaliação visual de linearidade:")
    sns.pairplot(X)
    plt.suptitle("Pairplot para avaliar linearidade", y=1.02)
    plt.show()


def rodar_LDA(X:pd.DataFrame, y:pd.DataFrame):
    print("\nRodando LDA (teste de classificação):")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    y_pred = lda.predict(X_test)

    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

Seleção de Variáveis

In [ ]:
X = df[["NU_NOTA_CH","NU_NOTA_CN","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]]

# Variável dependente (classe de discriminação)
y = df["TP_DEPENDENCIA_ADM_ESC"]  # altere se quiser classificar por outro atributo

Testes de Pressupostos

In [ ]:
print("=== VERIFICAÇÃO DOS PRESSUPOSTOS DA LDA ===")

# Normalidade
testar_normalidade_univariada(X)
testar_normalidade_multivariada(X.sample(300))

# Linearidade (visual)
testar_linearidade(X)

# Outliers
outliers = testar_outliers(X)

# Multicolinearidade
testar_multicolinearidade(X)

# Homogeneidade
testar_homogeneidade(X, y)

Rodar LDA

In [ ]:
rodar_LDA(X, y)